# Reports

In [12]:
import os
import re
from datetime import datetime
current_datetime = datetime.now().strftime('%Y%m%d.%H%M')

import numpy as np
import pandas as pd

# for the notebook rendering 
from IPython.display import display, HTML, FileLink, Markdown
from IPython.display import FileLink as FL
from IPython.display import Markdown as MD


DIR = os.getcwd()
print(f'{DIR=}')

RDIR = os.path.join(DIR,'reports')
print(f'{RDIR=}')

DIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor'
RDIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor\\reports'


In [13]:
template_path = os.path.join(DIR,'template.html')

In [14]:
def break_down_buckets(idf,column,buckets,message='',nan_value=-1):
    """
    breaks a column down into buckets/bins
    """
    idf = idf.fillna(nan_value)

    print('',message,'\ncolumn: ',column, '\nbuckets: ', buckets)
    
    idf = pd.DataFrame(idf[column])
    idf['bucket'] = pd.cut(idf[column], bins=buckets)
    idf = idf.groupby(by='bucket').count()
    idf['percent'] = (idf[column]/idf[column].sum())*100
    idf['percent'] = idf['percent'].round(2)
    display(idf)

    return idf

#test
# break_down_buckets(df,'AGE_YRS',[-1,0,15,25,35,45,55,65,75,85,500])

In [15]:
def bar(num,denom=100.0,length=30,fillchar='█',emptychar='░'):
    fillnum = ((int)( (num/denom) * length))
    return '' + ( fillnum * fillchar ).ljust(length,emptychar)  + '' # + f" {(num/denom)*100.0:.2f}%     " 

In [16]:
# YYYYMMDDHHmm
DateFilter = None
DateFilter = 202501100000 # new firmware r44715
# DateFilter = 202501160000 # changed some settings in Error 404 NH
# DateFilter = 202503000000
# DateFilter = 202503200000
DateFilter = 20250321000
# DateFilter = 202502080000

In [17]:
def getfirmware(ssid,dt):
    if 'NH' in ssid:
        
        if dt <= 202501100810:
            return "r58881"
        elif dt > 202501100811 and dt <= 202501102101:
            return "r44715"
        elif dt > 202501102101 : #and dt <= 203000000000:
            return "r58881"
    else:
        return ""

In [18]:
data = {}

for i in os.listdir(RDIR):
    # print(i)
    try:

        text = ''
        with open(os.path.join(RDIR, i), 'r') as f:
            text = f.read()
        # print(text)

        dt = re.findall('Datetime_alt: .*\n',text)[0]
        dt = dt.replace('Datetime_alt: ','').replace('\\n','')
        dt = int(dt)
            
        if DateFilter != None:
            if dt < DateFilter:
                #move on to next
                print('exclude',dt,DateFilter)
                continue
            else:
                print('include',dt,DateFilter)

        wifi = re.findall('^SSID:.*\n',text)[0]
        wifi = wifi.replace('\n', '').replace('SSID: ','')
        # wifi = wifi + " " + getfirmware(wifi,dt)

        pings = re.findall('.*time=.*\n',text)
        pings = [ re.sub('.*time=','',p) for p in pings]
        pings = [ re.sub('ms.*\n','',p) for p in pings]
        pings = [ int(p) for p in pings]
        # print(*pings,sep='\n')


        signalstrength =  None
        try:
            signalstrength = re.findall('Signal_Strength:.*\n',text)[0]
            signalstrength = signalstrength.replace('\n', '').replace('Signal_Strength: ','').replace('%','')
        except:
            pass

        timeouts = re.findall('Request timed out.\n',text)

        trycount = re.findall('(Request|Reply).*\n',text)

        if wifi in data.keys():
            pass
        else:
            data[wifi] = {}
            data[wifi]['pings'] = []
            data[wifi]['timeouts'] = []
            data[wifi]['trycount'] = []
            data[wifi]['signalstrength'] = []
        
        data[wifi]['pings'].extend(pings)
        data[wifi]['timeouts'].extend([len(timeouts)])
        data[wifi]['trycount'].extend([len(trycount)])

        if signalstrength != None:
            data[wifi]['signalstrength'].append(int(signalstrength))

    except Exception as e:
        pass
        print(i,e)


Error404NH_202412292039.txt list index out of range
include 202412300625 20250321000
include 202412310635 20250321000
include 202412311659 20250321000
include 202501011257 20250321000
include 202501020539 20250321000
include 202501030416 20250321000
include 202501041159 20250321000
include 202501051039 20250321000
include 202501060604 20250321000
include 202501070520 20250321000
include 202501080600 20250321000
include 202501090551 20250321000
include 202501100810 20250321000
include 202501110123 20250321000
include 202501121229 20250321000
include 202501130644 20250321000
include 202501140522 20250321000
include 202501150633 20250321000
include 202501152246 20250321000
include 202501152256 20250321000
include 202501160552 20250321000
include 202501170804 20250321000
include 202501171155 20250321000
include 202501171205 20250321000
include 202501171329 20250321000
include 202501180755 20250321000
include 202501191342 20250321000
include 202501201050 20250321000
include 202501210523 202

In [19]:
# file = os.path.join(DIR,'docs','index.html')
# with open(file,'w',encoding='utf8') as f:

#     f.write(f'<h>{current_datetime}</h>')

#     for k in data.keys():
        
#         f.write('<p>' + '-'*20 + '</p>')

#         f.write(f'<p>{k}</p>')
#         df_pings = pd.DataFrame(data[k]['pings'])

#         csvlines = df_pings.describe().to_csv()
#         f.write(csvlines[4::].replace('\n','<br \>').replace(',','\t'))

#         f.write('<br \>')
#         f.write(f'Signal Strength (min) {min(data[k]["signalstrength"])}<br \>')
#         f.write(f'Signal Strength (mean) {sum(data[k]["signalstrength"])/len(data[k]["signalstrength"]):.2f}<br \>')
#         f.write(f'Signal Strength (max) {max(data[k]["signalstrength"])}<br \>')

#         # print(df_pings)

#         bdb = break_down_buckets(df_pings,0,[0,5,10,15,20,30,40,50,500])
#         bdb['bar'] = bdb.percent.apply(bar)
#         bdb = bdb.reset_index()
        

#         # display(bdb)

#         f.write('<br \>')
#         f.write(bdb.to_html(index=False))
#         f.write('<br \>')

#         f.write(f'Request timed out (failed pings):<br \>')
#         f.write(f'{ sum(data[k]['timeouts']) } out of { sum(data[k]['trycount']) }<br \>')
#         f.write(f'{ sum(data[k]['timeouts']) / sum(data[k]['trycount']) }')

In [20]:
segment = """
<h2 class=""><i class="nf nf-md-wifi"></i>Wifi: {%segment_header%}</h2>
<h3 class="">Signal Strength</h2>

    <div class="progress bg-dark bar-boarder" style="height: 25px;">
        <div class="progress-bar bg-dark " role="progressbar" style="width: {%zeromin%}%" aria-valuemin="0" aria-valuemax="100"></div>
        <div class="progress-bar rounded-start-2 bg-signalstrength-1 fw-semibold" role="progressbar" style="width: {%minmean%}%" aria-valuemin="0" aria-valuemax="100">{%ssmin%}%</div>
        <div class="progress-bar  bg-signalstrength-2 fw-semibold" role="progressbar" style="width: {%meanmean%}%" aria-valuemin="0" aria-valuemax="100">{%ssmean%}%</div>
        <div class="progress-bar rounded-end-2 bg-signalstrength-1 fw-semibold" role="progressbar" style="width: {%meanmax%}%" aria-valuemin="0" aria-valuemax="100">{%ssmax%}%</div>
    </div>

<br />
<h3 class="">Ping Table</h2>
{%ping_table%}

<div class="divider"></div>
"""

In [21]:
template = ''
with open(template_path, 'r', encoding='utf8') as f:
    template = f.read()

template = template.replace('{%report_header%}',str(current_datetime))

segments = ''
for k in data.keys():
    ns = segment

    ns = ns.replace('{%segment_header%}',k)

    ssmin = min(data[k]["signalstrength"])
    ssmean = sum(data[k]["signalstrength"])/len(data[k]["signalstrength"])
    ssmax = max(data[k]["signalstrength"])

    ns = ns.replace('{%zeromin%}',f'{ssmin:.2f}')
    ns = ns.replace('{%minmean%}',f'{ssmean-ssmin-2:.2f}')
    ns = ns.replace('{%meanmean%}',f'{4:.2f}')
    ns = ns.replace('{%meanmax%}',f'{ssmax-ssmean-2:.2f}')

    ns = ns.replace('{%ssmin%}',f'{ssmin:.0f}')
    ns = ns.replace('{%ssmean%}',f'{ssmean:.0f}')
    ns = ns.replace('{%ssmax%}',f'{ssmax:.0f}')

    df_pings = pd.DataFrame(data[k]['pings'])
    bdb = break_down_buckets(df_pings,0,[0,5,10,15,20,30,40,50,500])
    # bdb = break_down_buckets(df_pings,0,range(0,100,5))
    bdb['bar'] = bdb.percent.apply(bar)
    bdb = bdb.reset_index()

    # ns = ns.replace('{%ping_table%}',bdb.to_html(index=False,justify='left'))
    # ns = ns.replace('\"dataframe\"','\"table table-dark table-striped\"')

    failed_pings = sum(data[k]['timeouts'])
    failed_percent = f"{failed_pings/sum(data[k]['trycount']):.0f}"

    tbl = ''
    tbl += '<table class="table table-dark table-stripedx">\n'
    tbl += """
    <thead>
        <tr style="text-align: left;">
        <th>bucket</th>
        <th></th>
        </tr>
    </thead>
    <tbody>
    """
    for i,row in bdb.iterrows():
        print(i)
        # bucket = row['bucket']
        bucket = re.sub(r'[\(\]]', '', str(row['bucket']))
        bucket = re.sub(r',', '-', bucket)
        bucket = re.sub(r' ', '', bucket)
        tbl += f"""
            <tr>
            <td class="col-1">{bucket}</td>
            <td class="col-12">
                <div class="progress bar-boarder bg-dark">
                <div class="progress-bar overflow-visible bar-ping rounded" style="width: {row['percent']}%; height:25px">
                    <span class="badge text-start">
                    {row[0]} | {row['percent']}%
                    </span>
                </div>
                </div>
            </td>
            </tr>
        """
    tbl += f"""
        <tr>
        <td class="col-1 text-danger">Failed</td>
        <td class="col-12">
            <div class="progress bar-boarder-danger bg-dark">
            <div class="progress-bar overflow-visible bg-danger  rounded" style="width: {failed_percent}%; height:25px">
                <span class="badge-failed text-start">
                {failed_pings} | {failed_percent}%
                </span>
            </div>
            </div>
        </td>
        </tr>
    """
    tbl += """
    </table>
    </tbody>
    """

    ns = ns.replace('{%ping_table%}',tbl)

    segments += ns

template = template.replace('{%segments%}',segments)

file = os.path.join(DIR,'docs','index.html')
with open(file,'w',encoding='utf8') as f:
    f.write(template)

  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 500]


C:\Users\JGarza\AppData\Local\Temp\ipykernel_5736\3656564501.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()


,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",1790,15.94
"(10, 15]",6311,56.20
"(15, 20]",874,7.78
"(20, 30]",565,5.03
"(30, 40]",376,3.35
"(40, 50]",253,2.25
"(50, 500]",1060,9.44


0
1
2
3
4
5
6
7
  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 500]


C:\Users\JGarza\AppData\Local\Temp\ipykernel_5736\3656564501.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()


,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",5293,44.66
"(10, 15]",5194,43.82
"(15, 20]",528,4.45
"(20, 30]",472,3.98
"(30, 40]",105,0.89
"(40, 50]",51,0.43
"(50, 500]",210,1.77


0
1
2
3
4
5
6
7
  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 500]


C:\Users\JGarza\AppData\Local\Temp\ipykernel_5736\3656564501.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()


,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",269,3.02
"(10, 15]",5817,65.30
"(15, 20]",1722,19.33
"(20, 30]",634,7.12
"(30, 40]",165,1.85
"(40, 50]",92,1.03
"(50, 500]",209,2.35


0
1
2
3
4
5
6
7
